Detecting Corona VIrus Disease(COVID)-19 using Classification in Machine Learning

Import all the necessary libraries

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
# Set the path to your dataset
dataset_path = 'dataset'

In [ ]:
# Define the input shape of the CT scan images
input_shape = (224, 224, 3)

# Set the batch size and number of epochs
batch_size = 32
epochs = 10

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

In [ ]:
# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Build the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

# Save the model
model.save('covid_detection_model.h5')

We use the saved model to run test on some custom images

In [ ]:
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from PIL import Image

Preprocess the image and use the trained model to predict the status of Covid infection

In [ ]:
# Load the pre-trained model
model = load_model('covid_detection_model.h5')

# defining class names
class_names = ["covid","non_covid"]

# Choose a test image
img_path = 'test.png'
img = Image.open(img_path)

# Preprocess the image
img = img.resize((224, 224))
img = img.convert('RGB')
img = np.array(img)
img = img.astype('float32') / 255.0  # Normalize the pixel values to be between 0 and 1
img = np.expand_dims(img, axis=0) 

# Make a prediction
predictions = model.predict(img)
print(predictions)
threshold = 0.01  # Adjust the threshold as needed
class_index = 0
if predictions[0] > threshold:
    print("The custom image is classified as COVID-19 positive.")
    class_index = 0
else:
    print("The custom image is classified as COVID-19 negative.")
    class_index = 1
class_name = class_names[class_index]

# Show the image and predicted class name
plt.imshow(img[0])
plt.axis('off')
plt.title(f'Prediction: {class_name}')
plt.show()